# Ex3 - Raz Bareli

### Q1)

In [8]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df = pd.read_csv("ex3.csv")
df.isna().sum()


We'll take only the 'safe' features we want to work with:

In [10]:
df = df[['state','date','congressional_district','gun_type','participant_gender', 'n_killed']]

In order to work with the data, we have to fill null values, and do some data engineering as we did in previous exercises, so that's what we'll do first.

In [11]:
# fill null values with mode as in ex1:
for column in df:
    df[column] = df[column].fillna(df[column].mode()[0])

In [12]:
# modify 'participant_gender' as in ex2
df.loc[df['participant_gender'].str.contains('Female', regex=True) &                         df['participant_gender'].str.contains('Male', regex=True), ['participant_gender']] = "Both"
df.loc[df['participant_gender'].str.contains('Female', regex=True), ['participant_gender']] = "Female"
df.loc[df['participant_gender'].str.contains('Male', regex=True), ['participant_gender']] = "Male"


In [ ]:
# modify 'gun_type' as in ex2
df['Gun']=df.gun_type.str.extract('([A-Za-z]+|[0-9][mm]+)')
def combine_guns(x):
    if x in ["Handgun","9mm","0mm", "Win","Spl", "Spr"]:
        return 'Handgun'
    if x in ["Other", "Unknown"]:
        return 'Unknown'
    else:
        return 'Rifle'
df["Gun"] = df["Gun"].apply(lambda x:combine_guns(x))
df['gun_type'] = df['Gun']
df = df.drop(columns='Gun')

In [19]:
# modify date to year/month as in ex1
def delete_day(x):
    return x[:-3]
df['date'] = df['date'].apply(delete_day)

Now we can get to the model training part:

In [21]:
# create dummy variables
df = pd.get_dummies(df, columns=['state', 'date', 'gun_type', 'participant_gender'])

In [23]:
X = df.drop(columns=['n_killed'])
y = df['n_killed']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)